# Data Migration - Program A

In [1]:
# Dependencies
import pandas as pd
import datetime as dt
from datetime import date
import warnings
import numpy as np
warnings.filterwarnings('ignore')

## Prepare Data Migration for Patent Family

In [ ]:
# READING DATA for Patent from Program A
prg-A_patent_df = pd.read_excel("Program A_Sample Data for Patent.xlsx", encoding = "utf8", dtype=str)
prg-A_patent_df

In [ ]:
# We want to validate values between Patent Family and Priority Application
# Patent Family Format: [Patent Holder]_[Priority Application]
# Example: 
# Patent Family: Disney_US2000-112233
# Priority Application: US2000-112233

# Extract separately patent holder and priority application in 'Patent Family' column
fam = prg-A_patent_df['Patent Family']
priority_app_fam = []

for f in fam:
    if "_" in f:
        priority_app_fam.append(f.split("_")[1])
    else:
        priority_app_fam.append("")
        
prg-A_patent_df["Priority App_from Family"] = priority_app_fam
prg-A_patent_df["Priority App_from Family"] = prg-A_patent_df["Priority App_from Family"].str.strip()

# Validate priority application we get from patent family is the same with values in 'Earliest Priority Application' column
priority_app = prg-A_patent_df['Earliest Priority Application']

prg-A_patent_df.loc[prg-A_patent_df['Priority App_from Family'] == priority_app, 'Validate_Priority App'] = 'YES'
prg-A_patent_df.loc[prg-A_patent_df['Priority App_from Family'] != priority_app, 'Validate_Priority App'] = 'NO'

diff_priority_app_index = prg-A_patent_df.index[prg-A_patent_df['Validate_Priority App'] == 'NO'].tolist()

for i in diff_priority_app_index:
    prg-A_patent_df['Earliest Priority Application'][i] = prg-A_patent_df['Priority App_from Family'][i]
    print(prg-A_patent_df.iloc[i])

In [ ]:
# Replace patent number "20050873 (Superseded)" with "200500873 (Superseded)"
# prg-A_patent_df = prg-A_patent_df.replace(["20050873 (Superseded)"], "200500873 (Superseded)")
# prg-A_patent_df = prg-A_patent_df.replace(["ZA20050873 (Superseded)"], "ZA200500873 (Superseded)")

# Extract separately patent number and "Superseded" (if any) in 'Patent Number' column
patent_no = prg-A_patent_df['Patent Number'].astype(str)
patent_no_clean = []
superseded = []

for p in patent_no:
    if "(" in p:
        patent_no_clean.append(p.split("(")[0])
        superseded.append(p.split("(")[1])
    elif "_" in p:
        patent_no_clean.append(p.split("_")[0])
        superseded.append(p.split("_")[1])
    else:
        patent_no_clean.append(p)
        superseded.append("")
        
prg-A_patent_df["Patent Number_Clean"] = patent_no_clean
prg-A_patent_df["Patent Number_Clean"] = prg-A_patent_df["Patent Number_Clean"].str.strip()
prg-A_patent_df["Superseded?"] = superseded
prg-A_patent_df["Superseded?"] = prg-A_patent_df["Superseded?"].str.replace(')','')
prg-A_patent_df["Superseded?"] = prg-A_patent_df["Superseded?"].str.strip()

# Clean the same for 'Country and Patent Number' column
# Extract separately country & patent number and "Superseded" (if any) in 'Country and Patent Number' column
CP_no = prg-A_patent_df['Country and Patent Number'].astype(str)
CP_clean = []

for cp in CP_no:
    if "(" in cp:
        CP_clean.append(cp.split("(")[0])
    elif "_" in cp:
        CP_clean.append(cp.split("_")[0])
    else:
        CP_clean.append(cp)
        
prg-A_patent_df["Country and Patent No_Clean"] = CP_clean
prg-A_patent_df["Country and Patent No_Clean"] = prg-A_patent_df["Country and Patent No_Clean"].str.strip()

# The same cleaning for 'Parent Patent Number'
# Extract separately parent patent number and "Superseded" (if any) in 'Parent Patent Number' column

PP_no = prg-A_patent_df['Parent Patent Number'].astype(str)
PP_clean = []
PP_superseded = []

for pp in PP_no:
    if "(" in pp:
        PP_clean.append(pp.split("(")[0])
        PP_superseded.append(pp.split("(")[1])
    elif "_" in pp:
        PP_clean.append(pp.split("_")[0])
        PP_superseded.append(pp.split("_")[1])
    else:
        PP_clean.append(pp)
        PP_superseded.append("")
        
prg-A_patent_df["Parent Patent No_Clean"] = PP_clean
prg-A_patent_df["Parent Patent No_Clean"] = prg-A_patent_df["Parent Patent No_Clean"].str.strip()
prg-A_patent_df["Parent Patent Superseded?"] = PP_superseded
prg-A_patent_df["Parent Patent Superseded?"] = prg-A_patent_df["Parent Patent Superseded?"].str.replace(')','')
prg-A_patent_df["Parent Patent Superseded?"] = prg-A_patent_df["Parent Patent Superseded?"].str.strip()
prg-A_patent_df

In [ ]:
# Validate each unique 'Earliest Priority Application' only has one unique 'Earliest Priority Date'
priority_check = prg-A_patent_df.pivot_table(values='Earliest Priority Date', index='Earliest Priority Application', aggfunc=pd.Series.nunique)
priority_check = priority_check.reset_index()
# priority_check
fix = list(priority_check.loc[priority_check['Earliest Priority Date'] > 1]['Earliest Priority Application'])
for app in fix:
    print(app)
    print(prg-A_patent_df.loc[prg-A_patent_df['Earliest Priority Application'] == app]['Earliest Priority Date'])

In [ ]:
# Replace and update the 'Earliest Priority Date'
for i in range(0, len(prg-A_patent_df['Earliest Priority Application'])):
    if prg-A_patent_df['Earliest Priority Application'][i] == 'KR2007-0109729' and prg-A_patent_df['Earliest Priority Date'][i] == '2009-10-30 00:00:00':
        prg-A_patent_df['Earliest Priority Date'][i] = '2007-10-30 00:00:00'
    elif prg-A_patent_df['Earliest Priority Application'][i] == 'SE1999-03552' and prg-A_patent_df['Earliest Priority Date'][i] == '1999-01-10 00:00:00':
        prg-A_patent_df['Earliest Priority Date'][i] = '1999-10-01 00:00:00'
    elif prg-A_patent_df['Earliest Priority Application'][i] == 'SE2000-04163' and prg-A_patent_df['Earliest Priority Date'][i] == '2002-07-04 00:00:00':
        prg-A_patent_df['Earliest Priority Date'][i] = '2000-11-14 00:00:00'
    elif prg-A_patent_df['Earliest Priority Application'][i] == 'SE2000-04187' and prg-A_patent_df['Earliest Priority Date'][i] == '2001-11-15 00:00:00':
        prg-A_patent_df['Earliest Priority Date'][i] = '2000-11-15 00:00:00'
    elif prg-A_patent_df['Earliest Priority Application'][i] == 'US2009-213593' and prg-A_patent_df['Earliest Priority Date'][i] == '2009-12-23 00:00:00':
        prg-A_patent_df['Earliest Priority Date'][i] = '2009-06-23 00:00:00'
    elif prg-A_patent_df['Earliest Priority Application'][i] == 'US2009-145223' and pd.isnull(prg-A_patent_df['Earliest Priority Date'][i]) == True:
        prg-A_patent_df['Earliest Priority Date'][i] = '2009-01-16 00:00:00'

# Re-checking each unique priority application only has one unique priority date
priority_check_2 = prg-A_patent_df.pivot_table(values='Earliest Priority Date', index='Earliest Priority Application', aggfunc=pd.Series.nunique)
priority_check_2 = priority_check_2.reset_index()
check = list(priority_check_2.loc[priority_check_2['Earliest Priority Date'] > 1]['Earliest Priority Application'])
check

In [ ]:
prg-A_patent_df.loc[prg-A_patent_df['Earliest Priority Date'].isnull() == True]['Patent Family'].unique()

In [ ]:
# Checking the all family information of child patents are the same as the family information of parent patents
# Separately extract and filter parent/CEP patents and child/FC patents
CEP_patent_df = prg-A_patent_df.loc[prg-A_patent_df['Parent Patent Number'].isnull() == True]
FC_patent_df = prg-A_patent_df.loc[prg-A_patent_df['Parent Patent Number'].isnull() == False]

CEP_patent_filtered_df = CEP_patent_df[['Patent: Matter Number','Patent Number_Clean','Primary Patent Country',
                                       'Earliest Priority Application', 'Earliest Priority Date']]
FC_patent_filtered_df = FC_patent_df[['Patent: Matter Number','Patent Number_Clean','Primary Patent Country',
                                      'Earliest Priority Application', 'Earliest Priority Date',
                                      'Parent Patent No_Clean','Parent Patent']]
CEP_patent_filtered_df.columns = ['Matter Number_Parent','CEP_Patent Number_Clean','CEP_Primary Patent Country',
                                  'CEP_Earliest Priority Application', 'CEP_Earliest Priority Date']
FC_patent_filtered_df.columns = ['Matter Number','FC_Patent Number_Clean','FC_Primary Patent Country',
                                 'FC_Earliest Priority Application', 'FC_Earliest Priority Date',
                                 'FC_Parent Patent No_Clean','Matter Number_Parent']

# Merge/Join both dataframes
FC_PP_validation = pd.merge(FC_patent_filtered_df, CEP_patent_filtered_df, on='Matter Number_Parent', how='left')
print("Vouched Patent: {}".format(FC_patent_filtered_df.shape))
print("Certified Patent: {}".format(CEP_patent_filtered_df.shape))
print("Merged Data: {}".format(FC_PP_validation.shape))

# Validate the family information between child and parent patents
FC_priority_app = FC_PP_validation['FC_Earliest Priority Application']
FC_priority_date = FC_PP_validation['FC_Earliest Priority Date']
FC_PP = FC_PP_validation['FC_Parent Patent No_Clean']

CEP_priority_app = FC_PP_validation['CEP_Earliest Priority Application']
CEP_priority_date = FC_PP_validation['CEP_Earliest Priority Date']
CEP_patent_no = FC_PP_validation['CEP_Patent Number_Clean']

FC_PP_validation.loc[FC_PP == CEP_patent_no, 'Validate_FC-PP_Parent Patent No'] = 'YES'
FC_PP_validation.loc[FC_PP != CEP_patent_no, 'Validate_FC-PP_Parent Patent No'] = 'NO'

FC_PP_validation.loc[FC_priority_app == CEP_priority_app, 'Validate_FC-PP_Priority App'] = 'YES'
FC_PP_validation.loc[FC_priority_app != CEP_priority_app, 'Validate_FC-PP_Priority App'] = 'NO'

FC_PP_validation.loc[FC_priority_date == CEP_priority_date, 'Validate_FC-PP_Priority Date'] = 'YES'
FC_PP_validation.loc[FC_priority_date != CEP_priority_date, 'Validate_FC-PP_Priority Date'] = 'NO'

diff_PP_no = FC_PP_validation.loc[FC_PP_validation['Validate_FC-PP_Parent Patent No'] == 'NO']
diff_priority_app = FC_PP_validation.loc[FC_PP_validation['Validate_FC-PP_Priority App'] == 'NO']
diff_priority_date = FC_PP_validation.loc[FC_PP_validation['Validate_FC-PP_Priority Date'] == 'NO']

print(diff_PP_no)
print(diff_priority_app)
print(diff_priority_date)

In [ ]:
# Read Program A Patent List (Parent/CEP only) with PatBase Family ID
prg-A_w_Patbase_ID_raw = pd.read_excel("prg-A with PatBase Family ID_23Aug2020.xlsx", encoding = "utf8", dtype=str)
additional_data_raw = pd.read_excel("additional-data.xlsx", encoding = "utf8", dtype=str)
# These files also contains Patent Number that needs to be updated 

# Filter and rename columns
prg-A_w_Patbase_ID = prg-A_w_Patbase_ID_raw[['Patent Number','Patent Number (Need to Update)','PatBase Family ID']]
prg-A_w_Patbase_ID.columns = ['Patent Number_Clean','Patent Number (Need to Update)','PatBase Family ID']

additional_data_raw['Patent Number (Need to Update)'] = ''
additional_data = additional_data_raw[['Patent Number','Patent Number (Need to Update)','PatBase Family']]
additional_data.columns = ['Patent Number_Clean','Patent Number (Need to Update)','PatBase Family ID']

comb_w_Patbase_ID = pd.concat([prg-A_w_Patbase_ID, additional_data])

# Extract Parent/CEP and Child/FC Patents separately from prg-A_patent_df
CEP_patent_df = prg-A_patent_df.loc[prg-A_patent_df['Parent Patent Number'].isnull() == True]
FC_patent_df = prg-A_patent_df.loc[prg-A_patent_df['Parent Patent Number'].isnull() == False]

# Merge prg-A patent data with PatBase Family ID separately (certified merge with patent number, vouched merge with parent patent number)
# Certified
CEP_patent_df = pd.merge(CEP_patent_df, comb_w_Patbase_ID, on='Patent Number_Clean', how='left')

# Vouched
prg-A_w_Patbase_ID_as_parent = comb_w_Patbase_ID[['Patent Number_Clean','Patent Number (Need to Update)','PatBase Family ID']]
prg-A_w_Patbase_ID_as_parent.columns = ['Parent Patent No_Clean','Parent Patent Number (Need to Update)','PatBase Family ID']
FC_patent_df = pd.merge(FC_patent_df, prg-A_w_Patbase_ID_as_parent, on='Parent Patent No_Clean', how='left')

# Concat Certified and Vouched patent data 
prg-A_patent_concat = pd.concat([CEP_patent_df, FC_patent_df])
prg-A_patent_concat

In [ ]:
prg-A_patent_concat[prg-A_patent_concat[['Patent: Matter Number']].duplicated()==True]

In [ ]:
prg-A_patent_concat.loc[prg-A_patent_concat['PatBase Family ID'].isnull()==True]

In [ ]:
prg-A_patent_df = prg-A_patent_concat[['Patent Holder', 'Patent: Matter Number', 'Patent Number',
                                   'Publication Number', 'Primary Patent Country',
                                   'Patent Expiration Date', 'EA EP GC Counterparts', 'Patent Family',
                                   'Earliest Priority Application', 'Earliest Priority Date',
                                   'Wireless Innovation Partnership Group', 'Patent Status',
                                   'Joint Ownership Percentage', 'Country and Patent Number',
                                   'Essentiality Status', 'Parent Patent Number', 'Parent Patent',
                                   'Patent: ID', 'Priority App_from Family', 'Validate_Priority App',
                                   'Patent Number_Clean', 'Superseded?', 'Country and Patent No_Clean',
                                   'Parent Patent No_Clean', 'Parent Patent Superseded?',
                                   'PatBase Family ID', 'Patent Number (Need to Update)','Pool Exit Date']]
prg-A_patent_df = prg-A_patent_df.reset_index(drop=True)
prg-A_patent_df